In [1]:
import numpy as np
import torch
import torch.nn as nn
import time
import joblib

In [2]:
# --------- 1. Define the same model structure ---------
class GestureClassifier(nn.Module):
    def __init__(self, input_size=63, num_classes=5):  # <-- adjust num_classes if needed
        super(GestureClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# --------- 2. Load the original model ---------
input_size = 63
model = GestureClassifier(input_size=input_size, num_classes=joblib.load('label_encoder.pkl').classes_.shape[0])
model.load_state_dict(torch.load("gesture_classifier.pth"))
model.eval()

# --------- 3. Generate dummy input ---------
scaler = joblib.load('scaler.pkl')
dummy_np = np.random.rand(1, input_size).astype(np.float32)
dummy_scaled = scaler.transform(dummy_np)
dummy_tensor = torch.from_numpy(dummy_scaled)

# --------- 4. Benchmark original model ---------
def benchmark_model(model, input_tensor, runs=1000):
    with torch.no_grad():
        start_time = time.time()
        for _ in range(runs):
            _ = model(input_tensor)
        end_time = time.time()
    avg_time_ms = (end_time - start_time) / runs * 1000
    return avg_time_ms

original_time = benchmark_model(model, dummy_tensor)
print(f"🕒 Original model average inference time: {original_time:.4f} ms")

# --------- 5. Convert to TorchScript using tracing ---------
traced_model = torch.jit.trace(model, dummy_tensor)
torch.jit.save(traced_model, "gesture_classifier_traced.pt")
print("✅ TorchScript model saved as gesture_classifier_traced.pt")

# --------- 6. Load and benchmark TorchScript model ---------
optimized_model = torch.jit.load("gesture_classifier_traced.pt")
optimized_model.eval()

optimized_time = benchmark_model(optimized_model, dummy_tensor)
print(f"⚡ TorchScript model average inference time: {optimized_time:.4f} ms")

# --------- 7. Optional: Performance gain ---------
speedup = original_time / optimized_time
print(f"🚀 Speedup: {speedup:.2f}x")

🕒 Original model average inference time: 0.2072 ms
✅ TorchScript model saved as gesture_classifier_traced.pt
⚡ TorchScript model average inference time: 0.0932 ms
🚀 Speedup: 2.22x
